**Connnexion d'abord, sinon pas d'autocompletion**

### Colab keyboards shortcuts:


https://amitness.com/2020/06/google-colaboratory-tips/

https://stackoverflow.com/questions/52438415/is-it-possible-to-have-autocomplete-in-a-notebook-in-google-colab<br/>

Run the selected cell<br/>
Ctrl+Enter

Run the selected cell and pass to the next <br/>
shift+Enter

Authocomplete<br/>
Ctrl + Space

Create cell	Command<br/>
Ctrl+M+B

Delete cell	Command<br/>
Ctrl+M+D

Select all cells<br/>
Ctrl+Shift+A

to code<br/>
Ctrl+M+Y

to markdown<br/>
Ctrl+M+M

Run all cells<br/>
Ctrl+F9


Interrupt execution<br/>
Ctrl+M+I

Select Cell and press Esc to go to command mode. <br/>
Hit Shift + Up or Shift + Down to select multiple cells

In [ ]:
install.packages('tidymodels')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘Rcpp’, ‘diagram’, ‘lava’, ‘listenv’, ‘parallelly’, ‘prodlim’, ‘future’, ‘warp’, ‘iterators’, ‘lhs’, ‘DiceDesign’, ‘patchwork’, ‘globals’, ‘clock’, ‘gower’, ‘ipred’, ‘timeDate’, ‘furrr’, ‘slider’, ‘foreach’, ‘GPfit’, ‘modelenv’, ‘dials’, ‘hardhat’, ‘infer’, ‘modeldata’, ‘parsnip’, ‘recipes’, ‘rsample’, ‘tune’, ‘workflows’, ‘workflowsets’, ‘yardstick’




In [ ]:
library('tidymodels')
library('tidyverse')

In [ ]:
# déposer directement le fichier, non dossier, non zip
list.files()


[1] "readmission_avc.rds" "sample_data"

In [ ]:
dataini <- readRDS('readmission_avc.rds')
dim(dataini)
head(dataini)
glimpse(dataini)

In [ ]:
glimpse(dataini)

Rows: 1,700
Columns: 12
$ modeEntree <int> 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 8,…
$ modeSortie <int> 9, 8, 7, 8, 6, 9, 7, 8, 8, 8, 8, 6, 7, 8, 7, 8, 8, 6, 8, 6,…
$ duree      <int> 0, 3, 13, 11, 8, 18, 8, 8, 14, 0, 5, 10, 1, 4, 20, 4, 11, 8…
$ ghm2       <chr> "01M37E", "01C061", "01M303", "01M301", "01M303", "01M303",…
$ dp         <chr> "I671", "I652", "I634", "I639", "I635", "I635", "I639", "I6…
$ sexe       <int> 2, 2, NA, 1, 1, NA, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, …
$ age        <int> 76, 77, NA, 83, 71, NA, 92, 88, 83, 82, 75, 83, 47, 76, 78,…
$ nbActe     <int> 4, 4, 4, 4, 4, 9, 7, 8, 7, 5, 6, 3, 1, 4, 30, 0, 7, 3, 6, 9…
$ nbRum      <int> 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 3, 1, 1, 2, 2, 1, 2, 2, 1, 3,…
$ nbda       <int> NA, 1, 7, 2, 9, 6, 4, 5, 4, 3, 3, 3, NA, 1, 8, 5, 3, 5, 10,…
$ id         <chr> "l19", "s7e", "23f", "8oi", "otz", "bof", "np7", "pu6", "t0…
$ id_D       <chr> "", "", "", NA, "ld", NA, "", "", NA, "", "", "9s", "", "",…


### Data description

a.	ident = un identifiant pour chaque séjour, id_D est le identifiant du séjour suivant

b.	Ghm2 = Groupage GENRSA, une sorte de classification pour les séjours hospitaliers, décisive pour leur remboursement

c.	DP = Diagnostic principal, le motif médical d'hospitalisation

d.	nbActe = Nombre de zones d'actes (acte = geste médicale : radio, écho, irm...) dans ce séjours
data

e.	nbRum = Nombre d’Unités Médicales (services à l’hôpital) contenus dans ce séjours

f.	nbda = Nombre de diagnostics associés dans ce séjour (les comorbilités)

g.	nbSeance = Nombre de séances ()

h. modeEntree / modeSortie : <br/>
* 6 = mutation depuis/vers un autre service
* 7 = transfert depuis/vers un autre
* 8 = domicile
* 9 = décès

i. duree = duree de séjour (en jour)


### Exploratory data analysis
* checking missing values in all columns
* distribution tables and plots

In [ ]:
is.na(dataini)
colSums(is.na(dataini))

modeEntree modeSortie      duree       ghm2         dp       sexe        age 
         0          0          0          0          0         20         20 
    nbActe      nbRum       nbda         id       id_D 
         0          0        134          0        200

In [ ]:
apply(is.na(dataini), 2, sum) # function at last position

modeEntree modeSortie      duree       ghm2         dp       sexe        age 
         0          0          0          0          0         20         20 
    nbActe      nbRum       nbda         id       id_D 
         0          0        134          0        200

In [ ]:
# map, appliquer la fonction a chaque colonne (vector) 1er argument est un DF
map(dataini, function(x) sum(is.na(x))) # => x ici est chaque colonne (un vector)
map(dataini, ~sum(is.na(.)))
dataini%>%map(~sum(is.na(.)))

### 0. Reconding and data preprocessing

In [ ]:
ls()
head(dataini)

ERROR: Error in eval(expr, envir, enclos): object 'dataini' not found


In [ ]:
dataset <-
dataini%>%
  filter(!is.na(id_D))%>%
  mutate(target = as.factor(
    if_else(id_D == '', 0, 1) # ==
    ))%>%
  mutate_at(c('modeEntree', 'modeSortie', "sexe"), as.character)%>% # as.character or as.factor
# Affects only the columns that were explicitly defined in the function, mutate_if() VS mutate_all()
  mutate(nbda = if_else(is.na(nbda), 0, nbda))%>%
  filter(modeSortie !=9)%>%
  select(-c(id, id_D))

dataset %>% group_by(nbda) %>% summarise(n = n())

# dataset[is.na(dataset$nbda), 'nbda'] <- 0

dataset %>%
  group_by(target)%>%
  summarise(n = n())

dim(dataset)

[1] 1322   11

### 1. Resample library : training set, validation set and test set

In [ ]:
set.seed(24) # 42 give metrics too high at the begining
dataset_split <- initial_split(dataset, strata = target, prop = 0.8)
class(dataset_split)
class(dataset)

training <- training(dataset_split)
test_set <- testing(dataset_split)

set.seed(42) # necessary if not executed with 'set.seed(24)' before
training_split <- initial_split(training, strata = target, prop = 0.8)
train_set = training(training_split)
eval_set = testing(training_split)
head(train_set, 50)
dim(train_set)
dim(eval_set)
dim(test_set)

cat(dim(train_set), '\t', dim(eval_set),'\t',dim(test_set))

### 2. Recipes |reh·suh·peez| library : create a collection of input formulas
#### 2.1 Basic recipe and visualisation of design matrix

In [ ]:
rec_basic <- recipe(data = train_set, target~.)
rec_basic



── Recipe ──────────────────────────────────────────────────────────────────────



── Inputs 

Number of variables by role

outcome:    1
predictor: 10



In [ ]:
prep(rec_basic) # estimate the required parameters from a training set that can be later applied to other data sets prep() returns an updated recipe with the estimates
juice(prep(rec_basic)) # design matrix
formula(prep(rec_basic))

target ~ modeEntree + modeSortie + duree + ghm2 + dp + sexe + 
    age + nbActe + nbRum + nbda
<environment: 0x5b24c5e47f30>

* REF : https://stackoverflow.com/questions/67079332/in-building-a-predictive-model-in-r-does-using-step-normalize-function-tidymod

* Furture reading: https://www.tmwr.org/recipes.html



The idea of a recipe is that you estimate statistics from the training set and then apply that same preprocessing to any other data, like the testing set or new data at prediction time.

In [ ]:
rec_basic <- recipe(train_set, target~.)%>%
  step_impute_mean(age)%>%
  step_impute_mode(sexe)%>%
  step_normalize(age)%>%
  step_dummy(modeEntree)%>%
  # step_dummy(dp)%>%
  step_other(dp, threshold= 0.05)%>% # first other() then dummy()
  step_dummy(dp)


juice(prep(rec_basic))

colnames(dataset)

[1] "modeEntree" "modeSortie" "duree"      "ghm2"       "dp"        
 [6] "sexe"       "age"        "nbActe"     "nbRum"      "nbda"      
[11] "target"

In [ ]:
# More general version
rec_basic <- recipe(data = train_set, target~.)%>%
  step_impute_mean(all_numeric_predictors())%>%
  step_impute_mode(all_nominal_predictors())%>%
  step_normalize(all_numeric_predictors())%>%
  step_other(dp, threshold = 0.05)%>%
  step_other(ghm2, threshold = 0.02)%>% # without step other metric is less, and some value not in training set
  step_dummy(all_nominal_predictors())

colnames(juice(prep(rec_basic)))

dim(juice(prep(rec_basic)))

[1] "duree"         "age"           "nbActe"        "nbRum"        
 [5] "nbda"          "target"        "modeEntree_X7" "modeEntree_X8"
 [9] "modeSortie_X7" "modeSortie_X8" "ghm2_X01C062"  "ghm2_X01M301" 
[13] "ghm2_X01M302"  "ghm2_X01M303"  "ghm2_X01M304"  "ghm2_X01M30T" 
[17] "ghm2_X01M311"  "ghm2_X01M312"  "ghm2_X01M31T"  "ghm2_other"   
[21] "dp_I634"       "dp_I635"       "dp_I638"       "dp_I639"      
[25] "dp_I652"       "dp_other"      "sexe_X2"

[1] 845  27

####2.2 Interacting recipe

In [ ]:
rec_interaction <-
  rec_basic %>%
    step_interact(~age : duree)%>% # one more predictor cause age and duree are numeric
    # step_interact(~age:dp) %>% # ask why do not work
    step_interact(~age : starts_with('dp'))


colnames(juice(prep(rec_interaction)))

dim(juice(prep(rec_interaction)))

[1] "duree"          "age"            "nbActe"         "nbRum"         
 [5] "nbda"           "target"         "modeEntree_X7"  "modeEntree_X8" 
 [9] "modeSortie_X7"  "modeSortie_X8"  "ghm2_X01C062"   "ghm2_X01M301"  
[13] "ghm2_X01M302"   "ghm2_X01M303"   "ghm2_X01M304"   "ghm2_X01M30T"  
[17] "ghm2_X01M311"   "ghm2_X01M312"   "ghm2_X01M31T"   "ghm2_other"    
[21] "dp_I633"        "dp_I634"        "dp_I635"        "dp_I638"       
[25] "dp_I639"        "dp_I652"        "dp_other"       "sexe_X2"       
[29] "age_x_duree"    "age_x_dp_I633"  "age_x_dp_I634"  "age_x_dp_I635" 
[33] "age_x_dp_I638"  "age_x_dp_I639"  "age_x_dp_I652"  "age_x_dp_other"

[1] 845  36

####2.3 Spline recipe

In [ ]:
rec_spline <-
  rec_basic %>%
  step_ns(age, duree, deg_free = 3)
  # step_ns(age, duree, deg_free = tune()) # this would not allow to show the design matrix
colnames(juice(prep(rec_spline)))

dim(juice(prep(rec_spline)))

[1] "nbActe"        "nbRum"         "nbda"          "target"       
 [5] "modeEntree_X7" "modeEntree_X8" "modeSortie_X7" "modeSortie_X8"
 [9] "ghm2_X01C062"  "ghm2_X01M301"  "ghm2_X01M302"  "ghm2_X01M303" 
[13] "ghm2_X01M304"  "ghm2_X01M30T"  "ghm2_X01M311"  "ghm2_X01M312" 
[17] "ghm2_X01M31T"  "ghm2_other"    "dp_I633"       "dp_I634"      
[21] "dp_I635"       "dp_I638"       "dp_I639"       "dp_I652"      
[25] "dp_other"      "sexe_X2"       "age_ns_1"      "age_ns_2"     
[29] "age_ns_3"      "duree_ns_1"    "duree_ns_2"    "duree_ns_3"

[1] 845  32

### 3. Parsnip library : creating and fitting a model
#### 3.1 Logistical model and workflow

A model needs
* a declaration of algo
* an engine (implementation package of algo)
* a mode (classification / regression)

In [ ]:
log_mod <- logistic_reg()%>%
  set_engine('glm')%>%
  set_mode('classification')

A workflow needs
* a declaration of wf
* a recipe
* a model

In [ ]:
log_wf <- workflow()%>%
  add_recipe(rec_basic)%>%
  # add_recipe(rec_inter)%>%
  # add_recipe(rec_spline)%>%
  add_model(log_mod)

# fit(log_wf, train_set)

# equivalent à
log_fit <-log_wf %>%
  fit(train_set)

predict(log_fit, eval_set)
# WHY there is non preprocessing of eval_set ? log_fit(wf fitted applique all transformation of train_set to eval_set)
predict(log_fit, eval_set, type = "prob")

log_pred <- eval_set%>%
  select(target)%>%
  bind_cols(
    predict(log_fit, eval_set),
    predict(log_fit, eval_set, type = "prob")
  )

# colnames(log_pred)

head(log_pred, 20)

In [ ]:
log_wf <- workflow()%>%
  add_recipe(rec_basic)%>%
  # add_recipe(rec_inter)%>%
  # add_recipe(rec_spline)%>%
  add_model(log_mod)

log_fit <- fit(log_wf, train_set)

# predict(log_fit, eval_set)
# predict(log_fit, eval_set, type = 'prob')

log_pred <- eval_set %>%
  select(target)%>%
  bind_cols(predict(log_fit, eval_set),
            predict(log_fit, eval_set, type = 'prob')
            )

head(log_pred, 20)

3.2 Random forest

RF : https://medium.com/analytics-vidhya/random-forest-classifier-and-its-hyperparameters-8467bec755f6

Decision trees having low bias and high variance tends to overfit the data. So bagging technique becomes a very good solution for decreasing the variance in a decision tree.

In general, the more trees in the forest the more robust the forest looks like. In the same way in the random forest classifier, the higher the number of trees in the forest gives the high the accuracy results.

mtry 	=> *nb de feuilles, n'est pas implémenter dans tidymodels*
A number for the number (or proportion) of predictors that will be randomly sampled at each split when creating the tree models (specific engines only)

trees
An integer for the number of trees contained in the ensemble.

min_n => *profondeur d'arbre*
An integer for the minimum number of data points in a node that is required for the node to be split further.
= min_sample_split (in python): Parameter that tells the decision tree in a random forest the minimum required number of observations in any given node to split it. Default = 2

In [ ]:
install.packages('ranger')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘RcppEigen’




In [ ]:
rf_mod <-
  rand_forest()%>%
  # rand_forest(trees = 1000)%>%
  # tuning lets see at 6
  set_engine('ranger')%>%
  set_mode('classification')

In [ ]:
rf_wf <-
  workflow()%>%
  add_recipe(rec_basic)%>%
  # add_recipe(rec_spline)%>%
  add_model(rf_mod)

set.seed(24) # fais une fois sans une fois avec
rf_fit <-
  rf_wf%>%
    fit(train_set)

rf_pred <-
  eval_set%>%
    select(target)%>%
    bind_cols(
      predict(rf_fit, eval_set),
      predict(rf_fit, eval_set, type = 'prob')
    )

rf_pred

#### 3.3 xgboost

In [ ]:
install.packages('xgboost')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
xg_mod <-
  boost_tree()%>%
    set_engine('xgboost')%>%
    set_mode('classification')

xg_mod_tune <-
  boost_tree(mtry = tune(), trees= tune(),  min_n = tune())%>% # min_n= profondeur d'arbre, mtry = nb de feuilles
    set_engine('xgboost')%>%
    set_mode('classification')

In [ ]:
xg_wf <-
  workflow()%>%
  add_recipe(rec_basic)%>%
  add_model(xg_mod)

xg_fit <-
  xg_wf%>%
    fit(train_set)

xg_pred <-
  eval_set%>%
    select(target)%>%
    bind_cols(predict(xg_fit, eval_set),
              predict(xg_fit, eval_set, type = 'prob')

  )

#### 3.4 Others models

In [ ]:
install.packages("kernlab")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
install.packages('kknn')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘igraph’




### 4. Yardstick library : Evaluate model performance

#### 4.1 Evaluate log model

https://yardstick.tidymodels.org/reference/roc_auc.html

Relevant Level<br/>
There is no common convention on which factor level should automatically be considered the "event" or "positive" result when computing binary classification metrics. In yardstick, the default is to use the first level. To alter this, change the argument event_level to "second" to consider the last level of the factor the level of interest. For multiclass extensions involving one-vs-all comparisons (such as macro averaging), this option is ignored and the "one" level is always the relevant result.

In [ ]:
# exemple for Relevant Level

data(two_class_example)

two_class_example


roc_auc(two_class_example, truth, Class1)
# indicates how well the probabilities from the positive classes are separated from the negative classes means that sensibility + specificity maximum, meaning that the couve is maximum
# une proba positive, une proba negative. Est ce que ces deux groupes de prob sont capable de déterminer dans quel classe qu'on prédit

roc_auc(two_class_example, truth, Class2, event_level = "second")

In [ ]:
accuracy(log_pred,target, .pred_class) # 0.9339623

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.9339623


In [ ]:
roc_auc(log_pred, target, .pred_0) # 0.8069411 vs 0.8096852 (inter) vs 0.8074253 (spline)

# OR
roc_auc(log_pred, target, .pred_1, event_level = "second")

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
roc_auc,binary,0.8069411


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
roc_auc,binary,0.8069411


#### 4.2 Evaluate rf model

In [ ]:
accuracy(rf_pred, target, .pred_class) # 0.9339623
roc_auc(rf_pred, target, .pred_0) # 0.8366425 vs 0.8338 (inter) vs 0.826 (spline)

table(rf_pred$target, rf_pred$.pred_class)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.9339623


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
roc_auc,binary,0.8366425


   
      0   1
  0 177   0
  1  14  21

#### 4.3 Evaluate xg model

In [ ]:
accuracy(xg_pred, target, .pred_class) # 0.9386792

roc_auc(xg_pred, target, .pred_0) # 0.834 vs 0.8316 (inter) vs 0.8078


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
roc_auc,binary,0.8221953


### 5. Cross validation
Until now, log has better auc, but with cv, things change...

In [ ]:
set.seed(24)
folds <- vfold_cv(training, v=5)

log_fit <- log_wf%>%
    # fit(train_set) # we did this
  fit_resamples( # instead of using fit(train_set), here we use fit_resamples() and do all the cv
    resamples = folds,
    metrics = metric_set(accuracy, roc_auc, f_meas),
    # control = control_grid(save_pred = TRUE) # this is necessary if we want to collect the prediction => but there is no need to collect prediction
  )

log_fit%>%collect_metrics() # 0.9337700/ 0.9617313/0.8212109

# question : est-il nessessaire de calculer les prédiction à cette étape ? => why we collected before ?

log_fit%>%collect_predictions() # this do not work




.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.9413082,5,0.008852863,Preprocessor1_Model1
f_meas,binary,0.9657030,5,0.005370410,Preprocessor1_Model1
roc_auc,binary,0.8334413,5,0.015949651,Preprocessor1_Model1


In [ ]:
set.seed(24) # necessary here because its a random
rf_fit <-
  rf_wf%>%
  fit_resamples( # instead of using fit(train_set), here we use fit_resamples() and do all the cv
    resamples = folds,
    metrics = metric_set(accuracy, roc_auc, f_meas),
    # control = control_grid(save_pred = TRUE) # this is necessary if we want to collect the prediction => but there is no need to collect prediction
  )
rf_fit%>%collect_metrics() # 0.9318743 / 0.9607195	/ 0.8242635

### 6. Hyperparameter tuning

#### 6.1 Random grid search : Space filling design

Let's make the thing clair : When tuning hyperparameter, even less if you use Random grid search, you aim is never to find exact values for your hyperparameters but find a model with a combinaison of hyperparameters more performent than if you do not tuning. For example you want to obtain a model pretrained to make predictions. *And believe me, there is more chance that you want to a pretained model than put the exact values of your hyper parameters.*

And be aware that you will get a better model with tune than without.

In [ ]:
# prepare a rf_wf with Hyperparameter tuning

rf_mod_tune <-
  # rand_forest()%>%
  # rand_forest(trees = 1000)%>%
  rand_forest(trees = tune(), min_n = tune())%>% # profondeur d'arbre
  set_engine('ranger')%>%
  set_mode('classification')

rf_wf_tune <-
  workflow()%>%
    add_recipe(rec_spline)%>%
    add_model(rf_mod_tune)


In [ ]:
set.seed(24)

rf_fit_tuning <-
  rf_wf_tune %>%
    tune_grid( # here we use tune_grid() rather than fit_resamples(). tune_grid is a cv with tuning
      resamples = folds,
      metrics = metric_set(accuracy, roc_auc),
      grid = 20
    )

rf_fit_tuning %>% collect_metrics()
# ask why 40 lines
# ask to find which values of hyperparameters get the best roc_auc
rf_fit_tuning %>% collect_metrics() %>% filter(.metric == 'roc_auc')  %>% arrange(desc(mean))
# trees = 150; 	min_n = 5 # roc_auc = 0.8279899

# equivalent to
rf_fit_tuning%>%select_best(metric = 'roc_auc')

trees,min_n,.config
<int>,<int>,<chr>
230,13,Preprocessor1_Model11


### 7. Workflowset

#### 7.1 Create a workflowset

In [ ]:
# First, without option name, grid = 2, show rank_results
# Then, with option name, change splie with df = tune(), only keep rf_mod_tune, grid = 5
wf_set <-
  workflow_set(
    preproc = list(
      basic = rec_basic,
      inter = rec_inter,
      spline = rec_spline
    ),
    models = list(
      log = log_mod,
      rf = rf_mod,
      rf_tune = rf_mod_tune,
      xg = xg_mod,
      xg_tune = xg_mod_tune
    )
  )
) # wfset instead of wf

#### 7.2 fitting a workflowset (5 min with grid = 20)

In [ ]:
set.seed(24)
res_wf_set <- # never foget to affect the variable. Waste of time !
  wf_set%>%
  workflow_map( # if wf_set, using workflow_map(fn = "tune_grid") instead of tune_grid
    resamples = folds,
    metrics = metric_set(accuracy, roc_auc),
    grid = 20, # keep 20, because rf is the only tune model. If not we can show tuning hyperparameter results
    fn = "tune_grid",
    # control = keep_pred
    verbose = T
  )

#### 7.3 Evaluation of all wf in a wf_set

In [ ]:
# why 138 row :  {3 (log) + 3 (rf) + 3 *20 (rf tune) + 3 xg + 3*20 (xg_tune)} * 2 metrics

rank_results(res_wf_set, rank_metric = 'roc_auc') # 0.8547824

### 8. Last fit and final prediction
8.1 Extract the best workflow of the wf_set

In [ ]:
best_wf <-
  res_wf_set %>%
    extract_workflow_set_result('spline_xg')%>% # also show a spline_rf
    select_best(metric ='roc_auc')
best_wf



#### 8.2 Last fit

In [ ]:
lastfit <- res_wf_set %>%
  extract_workflow('spline_xg')%>%
  finalize_workflow(best_wf)%>% # push the best wf
  last_fit(split = dataset_split)

lastfit%>%collect_metrics() # 0.8658988
lastfit%>%collect_predictions() # here we can collect our predictions because predictions are not only used to calculate metrics

In [ ]:
### 9. Others models

In [ ]:
install.packages("kernlab")
install.packages('kknn')

In [ ]:
svm_p_spec_tune <-
  svm_poly(cost = tune(), degree = tune()) %>%
  set_engine("kernlab") %>%
  set_mode("classification")


knn_tune <-
  nearest_neighbor(neighbors = tune())%>%
    set_engine('kknn')%>%
    set_mode('classification')

nnet_tune <-
  mlp(penalty = tune(), epochs = tune())%>%
    set_engine('nnet')%>%
    set_mode('classification')

In [ ]:
wf_set_other <-
  workflow_set(
    preproc = list(
      basic = rec_basic,
      inter = rec_interaction,
      spline = rec_spline
    ),
    models = list(
      log = log_mod,
      # rf = rf_mod,
      rf = rf_mod_tune,
      xg = xg_mod,
      svm = svm_p_spec_tune,
      knn = knn_tune,
      nnet = nnet_tune
    )
  )

In [ ]:
set.seed(24)

res_wf_set <-
  wf_set_other%>%
    workflow_map(
      resamples = folds,
      metrics = metric_set(accuracy, roc_auc),
      grid = 20,
      fn = "tune_grid",
      verbose=T
    )

In [ ]:
rank_results(res_wf_set, rank_metric = 'roc_auc') # 0.8359445

In [ ]:
best_wf <-
  res_wf_set_other %>%
    extract_workflow_set_result('spline_xg')%>% # also show a spline_rf
    select_best(metric ='roc_auc')
best_wf